pip install pyMuPDF

In [119]:
import fitz
from typing import List
from vectordb import Memory



In [120]:
def formatar(txt: str):
  return txt.replace('\n', '\n')

In [121]:
doc = fitz.open('./credenciamento.pdf')


def pdf_para__lista(doc):
  total_paginas = doc.page_count
  lista_conteudo = []

  for i in range(total_paginas):
    texto = doc.load_page(i).get_text('text')
    texto = formatar(texto)
    pagina = 'Página ' + str(i + 1) + ' - '
    lista_conteudo.append(pagina + texto)
  
  doc.close()
  return lista_conteudo

# pdf_para__lista(doc)


In [122]:
class DataChuk:
    def __init__(self, metadata, content) -> None:
        self.metadata = metadata
        self.content = content


class Document:
    def __init__(self):
        self.sections = []

    def add_section(self, section_name, page_number, content):
        section_data = {
            'meta_data': {'section_name': {section_name}, 'Pagina': {page_number}},
            'section_content': content
        }

        data = DataChuk({'section_name': {section_name}, 'Pagina': {page_number}}, content)
        self.sections.append(data)
    
    def __repr__(self):
        return f"DataChuk({self.sections})"

def extract_sections(pdf_path) -> List[DataChuk]:
    document = fitz.open(pdf_path)
    doc_object =  Document()
    current_section = None
    
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text = page.get_text("text")
        lines = text.split('\n')
        
        for line in lines:
            if is_section_header(line):
                current_section = line.strip()
                doc_object.add_section(current_section, page_num + 1, '')
            if current_section:
                doc_object.sections[-1].content += line + '\n'
    
    return doc_object

def is_section_header(line):
 
    return len(line) > 0 and (line.isupper() or line.endswith(':'))


doc_object = extract_sections(doc)


sections = doc_object.sections

sections[0].metadata



{'section_name': {'EDITAL DE CREDENCIAMENTO DE OPERADORAS DE CARTÃO'},
 'Pagina': {1}}

In [143]:
memory = Memory(chunking_strategy={"mode": 'sliding_window', "window_size": 300, "overlap": 8})






Initiliazing embeddings:  normal


/Users/gabrieljesus/Documents/tcc/exprerimentos/env/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OK.


In [144]:
for i in range(len(sections)):
  capitulo = sections[i].content
  metadata = sections[i].metadata
  memory.save(capitulo, metadata)

In [169]:
memory.search('A qualificação técnica deve ser comprovada por meio de', top_n=5)

[{'chunk': 'SEFAZ/SE. 4.9 A qualificação técnica deve ser comprovada por meio de: 4.9.1 Certidão Negativa de Exercício de Administração em Instituição em Liquidação Extrajudicial, emitida pelo BACEN, dos ocupantes da função de direção, administração ou representação, em observância ao disposto na Lei Complementar federal nº 64, de 18 de maio de 1990, art. 1º, inciso I, alínea “i”, de que não se encontra em regime liquidação extrajudicial, obtida por meio do sítio eletrônico https://www3.bcb.gov.br/nadaconsta/emitirCertidaoRegesp; 4.9.2 habilitação para funcionar como subadquirente, com plena capacidade técnica e funcional para realizar a captura de dados dos DAE’s, e o processamento de pagamentos por meio eletrônico, mediante uso de cartões de crédito das principais bandeiras aceitos no mercado financeiro brasileiro; 4.9.3 demonstrativo que aderiu e está cumprindo as regras estabelecidas pelas bandeiras de cartão de crédito, através do contrato de participação nos arranjos de pagamento

In [3]:
# convert the document to markdown
import pymupdf4llm
md_text = pymupdf4llm.to_markdown("credenciamento.pdf", page_chunks=False, write_images=False)



# Write the text to some file in UTF8-encoding

pathlib.Path("output.md").write_bytes(md_text.encode())


# md_text[-1]

98898

'I'